In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import json
import os
import shutil
import random
from PIL import Image
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

print('Using PyTorch version:', torch.__version__)
if torch.cuda.is_available():
    print('Using GPU, device name:', torch.cuda.get_device_name(0))
    device = torch.device('cuda')
else:
    print('No GPU found, using CPU instead.')
    device = torch.device('cpu')

Using PyTorch version: 2.3.1
No GPU found, using CPU instead.


In [2]:
# Définir les transformations pour les données
transform = transforms.Compose([
    transforms.Resize((28, 28)),  # Redimensionner les images à 224x224
    transforms.ToTensor(),  # Convertir les images en tenseurs PyTorch
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normaliser les images (FashionMNIST est en niveaux de gris)
])

# Télécharger et charger le jeu de données FashionMNIST
dataset = datasets.FashionMNIST(root='./data', 
                                train=True, 
                                transform=transform, 
                                download=True)

# Séparer les données en ensemble d'entraînement et de test
# Calculer la taille de chaque sous-dataset
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# Diviser le dataset en ensemble d'entraînement et de test
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Créer des DataLoader pour les ensembles d'entraînement et de test
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Afficher les tailles des datasets pour vérification
print(f'Taille du dataset d\'entraînement: {len(train_dataset)}')
print(f'Taille du dataset de test: {len(test_dataset)}')

Taille du dataset d'entraînement: 48000
Taille du dataset de test: 12000


In [3]:
class SimpleMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 20),
            nn.ReLU(),
            nn.Linear(20, 10)
        )

    def forward(self, x):
        return self.layers(x)

model = SimpleMLP().to(device)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

def correct(output, target):
    predicted_digits = output.argmax(1)                            # choisir le chiffre avec la plus grande sortie du réseau
    correct_ones = (predicted_digits == target).type(torch.float)  # 1.0 pour correct, 0.0 pour incorrect
    return correct_ones.sum().item()                               # compter le nombre de bons

SimpleMLP(
  (layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=20, bias=True)
    (2): ReLU()
    (3): Linear(in_features=20, out_features=10, bias=True)
  )
)


In [4]:
def train(data_loader, model, criterion, optimizer):
    model.train()

    num_batches = len(data_loader)
    num_items = len(data_loader.dataset)

    total_loss = 0
    total_correct = 0
    for data, target in data_loader:
        # Copier les données et les cibles sur le GPU
        data = data.to(device)
        target = target.to(device)

        # Effectuer une passe avant
        output = model(data)

        # Calculer la perte
        loss = criterion(output, target)
        total_loss += loss

        # Compter le nombre de chiffres corrects
        total_correct += correct(output, target)

        # Rétropropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    train_loss = total_loss/num_batches
    accuracy = total_correct/num_items
    print(f"Average loss: {train_loss:7f}, accuracy: {accuracy:.2%}")

epochs = 10
for epoch in range(epochs):
    print(f"Training epoch: {epoch+1}")
    train(train_loader, model, criterion, optimizer)


Training epoch: 1
Average loss: 0.604730, accuracy: 79.32%
Training epoch: 2
Average loss: 0.443107, accuracy: 84.15%
Training epoch: 3
Average loss: 0.411755, accuracy: 85.31%
Training epoch: 4
Average loss: 0.391875, accuracy: 85.85%
Training epoch: 5
Average loss: 0.376635, accuracy: 86.40%
Training epoch: 6
Average loss: 0.364817, accuracy: 86.88%
Training epoch: 7
Average loss: 0.356330, accuracy: 87.17%
Training epoch: 8
Average loss: 0.347272, accuracy: 87.41%
Training epoch: 9
Average loss: 0.341421, accuracy: 87.66%
Training epoch: 10
Average loss: 0.334044, accuracy: 87.90%


In [5]:
def test(test_loader, model, criterion):
    model.eval()

    num_batches = len(test_loader)
    num_items = len(test_loader.dataset)

    test_loss = 0
    total_correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            # Copier les données et les cibles sur le GPU
            data = data.to(device)
            target = target.to(device)

            # Effectuer une passe avant
            output = model(data)

            # Calculer la perte
            loss = criterion(output, target)
            test_loss += loss.item()

            # Compter le nombre de chiffres corrects
            total_correct += correct(output, target)

    test_loss = test_loss/num_batches
    accuracy = total_correct/num_items

    print(f"Testset accuracy: {100*accuracy:>0.1f}%, average loss: {test_loss:>7f}")

test(test_loader, model, criterion)

Testset accuracy: 86.6%, average loss: 0.382256


In [ ]:
# Affichage de la matrice de confusion
conf_matrix = confusion_matrix(test_y,predict_y)
fig, ax = plt.subplots(figsize=(7, 7))  # Adjust the figsize as needed
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
disp.plot(cmap=plt.cm.Blues, ax=ax)
plt.show()
